In [ ]:
from aiida import load_profile, orm 
load_profile();

from aiida.orm import QueryBuilder
import ipywidgets as widgets
from IPython.display import display,clear_output

from settings import *
from utils import make_query

from IPython.core.display import display

# Inversion results

Remote directories of successful inversions. 

In [ ]:
extra_labels = widgets.Dropdown(
            options = make_query.get_extra_(INVERSION_WORKLFOW, name=None)
        )
extra_labels

In [ ]:
def get_data_inversions():
    qb = QueryBuilder()
    qb.append(
                INVERSION_WORKLFOW,
                tag = 'inv_w',
                filters={"attributes.exit_status": 0,
                         "extras": {"has_key": extra_labels.value}},
                project=["id"],
            ).append(
                INVERSION,
                tag = 'inv',
                with_incoming = 'inv_w', 
            )
    qb.append(orm.Dict,
            with_outgoing = 'inv',
            project = ['attributes.run_str','attributes.dtm_start','attributes.dtm_end'])
    qb.append(orm.RemoteStashFolderData, 
            with_incoming = 'inv',
            project = ['attributes.target_basepath'])
    return qb.all()

In [ ]:

def gen_table():
    html = """<style>
            table, th, td {
                border: 1px solid;
                text-align: center;
                padding: 5px}</style>
            <table>
                    <tr>
                        <th>Id</th>
                        <th>run.str</th>
                        <th>dtm.start</th>
                        <th>dtm.end</th>
                        <th>Remote address</th>
                    </tr>"""
    for i in get_data_inversions(): 
        html+=f"""  <td>{i[0]}</td>
                    <td>{i[1]}</td>
                    <td>{i[2]}</td>
                    <td>{i[3]}</td>
                    <td>{i[4]}</td>
                    </tr>"""
    html += "</table>"
    return html

In [ ]:
search_b = widgets.Button(description = 'Search',
                          button_style='info')
out = widgets.Output()
display(search_b,out)
def on_button_clicked(b):
    with out:
        clear_output()
        display(widgets.HTML(gen_table()))
    
search_b.on_click(on_button_clicked)